### shap 
shap word hier gebruikt om uit te legen wat dit model belangrijk vind
hiervoor gebruiken we de dataset die is aangepast met feature importens

In [1]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, make_scorer, precision_score, recall_score, classification_report
from sklearn.inspection import permutation_importance

#### bereid de data voor

In [2]:
all_data_selected = pd.read_csv('../../data/selected_data/all_data_selected.csv')
all_data_selected = all_data_selected.drop([ 'Unnamed: 0', 'full_text', 'user_description', 'username'], axis = 1)
# select x and y from data
y = all_data_selected.pop('real_fake_grade')
x = all_data_selected

# scale variables
x = StandardScaler().fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42, stratify=y)

#### maak de SVM

In [3]:
svm_rs = RandomizedSearchCV(SVC(), 
                        param_distributions = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                                            'C': [0.5, 1, 1.5, 2]},
                        cv = 7,
                        scoring = make_scorer(accuracy_score)) 

# fit the model
svm_rs.fit(x_train, y_train)
print('Best parameters:\n', svm_rs.best_params_)
print()

# make prediction with best parameters
y_pred = svm_rs.best_estimator_.predict(x_test)


Best parameters:
 {'kernel': 'rbf', 'C': 1.5}



#### shap maken

In [ ]:
from tqdm import tqdm
import shap
shap.initjs()

In [ ]:
explainer = shap.KernelExplainer(svm_rs.best_estimator_.predict,x_train)

In [ ]:
shap_values = explainer.shap_values(shap.sample(x_test, 25),nsamples=100)

In [ ]:
features = ['retweet_count',
            'user_followers_count',
            'user_friends_count',
            'user_favourites_count', 
            'user_statuses_count', 
            'user_media_count', 
            'hashtags_count',
            'text_length',
            'tweet_contains_url',
            'user_service_level_analytics',
            'user_service_level_dso',
            'user_service_level_media_studio',
            'user_service_level_reseller',
            'user_service_level_smb',
            'tweeted_in_weekend', 
            'user_created_in_weekend',
            'user_creation_tweet_diff', 
            'sent_via_twitter', 
            'twitter_android_user',    
            'possibly_sensitive_media',       
            'user_is_verified',
            'user_has_translation_enabled',          
            'user_has_default_profile',
            'user_has_default_profile_image',
            'user_has_custom_timeline',
            'has_pinned_tweet',
            'has_user_url',
            'tweet_contains_media',
            'tweet_is_reply',
            'user_is_advertiser',
            'part_of_thread',
            'user_is_advertiser',
            'user_decscription_has_urls',
            'tweeted_in_daypart_day',
            'tweeted_in_daypart_evening',
            'tweeted_in_daypart_morning',             
            'tweeted_in_daypart_night',   
            'user_created_in_daypart_day',      
            'user_created_in_daypart_evening',         
            ]


save de shap valeus als een np array in een csv

In [ ]:
from numpy import savetxt
savetxt('shap.csv', shap_values, delimiter=',')

shap inladen

In [ ]:
from numpy import loadtxt
# load array
shap_values_loaded = loadtxt('shap.csv', delimiter=',')

In [ ]:
shap.summary_plot(shap_values_loaded,x_test[:25],feature_names=features, show=False)
plt.gcf().axes[-1].set_aspect(100)
plt.gcf().axes[-1].set_box_aspect(10)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values_loaded[0],
    feature_names=features)